In [1]:
import pandas as pd
import numpy as np
import datetime

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Naive estimator

In [2]:
train = pd.read_csv('preprocessed_train.csv')
test = pd.read_csv('preprocessed_test.csv')

test.head()

,Unnamed: 0,index,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,...,day_week,time_of_day,position,prev_event,2prev_event,next_event,prev_time,next_time,seconds_next,seconds_prev
0,0,0,32422708117504,197219,2012-01-02 15:28:39.244,10000,112.0,A_SUBMITTED,COMPLETE,2012-01-02 15:28:39.244,...,0,15,1,FIRST EVENT,FIRST EVENT,A_PARTLYSUBMITTED,2012-01-02 15:28:39.244,2012-01-02 15:28:39.416,0.172,0.000
1,1,1,32422708117505,197219,2012-01-02 15:28:39.244,10000,112.0,A_PARTLYSUBMITTED,COMPLETE,2012-01-02 15:28:39.416,...,0,15,2,A_SUBMITTED,FIRST EVENT,A_PREACCEPTED,2012-01-02 15:28:39.244,2012-01-02 15:29:09.496,30.080,0.172
2,2,2,32422708117506,197219,2012-01-02 15:28:39.244,10000,112.0,A_PREACCEPTED,COMPLETE,2012-01-02 15:29:09.496,...,0,15,3,A_PARTLYSUBMITTED,A_SUBMITTED,W_Completeren aanvraag,2012-01-02 15:28:39.416,2012-01-02 15:29:09.906,0.410,30.080
3,3,3,32422708117507,197219,2012-01-02 15:28:39.244,10000,112.0,W_Completeren aanvraag,SCHEDULE,2012-01-02 15:29:09.906,...,0,15,4,A_PREACCEPTED,A_PARTLYSUBMITTED,W_Completeren aanvraag,2012-01-02 15:29:09.496,2012-01-02 19:41:59.956,15170.050,0.410
4,4,4,32422708117508,197219,2012-01-02 15:28:39.244,10000,10863.0,W_Completeren aanvraag,START,2012-01-02 19:41:59.956,...,0,19,5,W_Completeren aanvraag,A_PREACCEPTED,W_Completeren aanvraag,2012-01-02 15:29:09.906,2012-01-02 19:46:24.381,264.425,15170.050


In [3]:
def naive_time_estimator(train, test):
    train['next position'] = train.groupby('case concept:name')['position'].shift(-1) #Creates new column with NaN values for the last even in a case
    test['next position'] = test.groupby('case concept:name')['position'].shift(-1)
    
    train = train.dropna(subset=['next position']) #Drop all last events per case
    
    train_position = train.groupby('position')['seconds_next'].mean()
    print(train_position)
    test['prediction_seconds'] = test.apply(lambda x: train_position[x['position']]
                                            if not pd.isnull(x['next position']) else np.nan, axis = 1)
    
    return test

naive_time_estimator(train, test)

test.head(10)

position
1          0.585243
2         34.974184
3      10192.966870
4      12220.322218
5       1132.341762
           ...     
170       14.929000
171     1568.870000
172      304.951000
173        0.000000
174        3.406000
Name: seconds_next, Length: 174, dtype: float64


,Unnamed: 0,index,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,...,position,prev_event,2prev_event,next_event,prev_time,next_time,seconds_next,seconds_prev,next position,prediction_seconds
0,0,0,32422708117504,197219,2012-01-02 15:28:39.244,10000,112.0,A_SUBMITTED,COMPLETE,2012-01-02 15:28:39.244,...,1,FIRST EVENT,FIRST EVENT,A_PARTLYSUBMITTED,2012-01-02 15:28:39.244,2012-01-02 15:28:39.416,0.172,0.000,2.0,0.585243
1,1,1,32422708117505,197219,2012-01-02 15:28:39.244,10000,112.0,A_PARTLYSUBMITTED,COMPLETE,2012-01-02 15:28:39.416,...,2,A_SUBMITTED,FIRST EVENT,A_PREACCEPTED,2012-01-02 15:28:39.244,2012-01-02 15:29:09.496,30.080,0.172,3.0,34.974184
2,2,2,32422708117506,197219,2012-01-02 15:28:39.244,10000,112.0,A_PREACCEPTED,COMPLETE,2012-01-02 15:29:09.496,...,3,A_PARTLYSUBMITTED,A_SUBMITTED,W_Completeren aanvraag,2012-01-02 15:28:39.416,2012-01-02 15:29:09.906,0.410,30.080,4.0,10192.966870
3,3,3,32422708117507,197219,2012-01-02 15:28:39.244,10000,112.0,W_Completeren aanvraag,SCHEDULE,2012-01-02 15:29:09.906,...,4,A_PREACCEPTED,A_PARTLYSUBMITTED,W_Completeren aanvraag,2012-01-02 15:29:09.496,2012-01-02 19:41:59.956,15170.050,0.410,5.0,12220.322218
4,4,4,32422708117508,197219,2012-01-02 15:28:39.244,10000,10863.0,W_Completeren aanvraag,START,2012-01-02 19:41:59.956,...,5,W_Completeren aanvraag,A_PREACCEPTED,W_Completeren aanvraag,2012-01-02 15:29:09.906,2012-01-02 19:46:24.381,264.425,15170.050,6.0,1132.341762
5,5,5,32422708117509,197219,2012-01-02 15:28:39.244,10000,10863.0,W_Completeren aanvraag,COMPLETE,2012-01-02 19:46:24.381,...,6,W_Completeren aanvraag,W_Completeren aanvraag,W_Completeren aanvraag,2012-01-02 19:41:59.956,2012-01-03 18:05:28.583,80344.202,264.425,7.0,12659.842273
6,6,22,32422708117510,197219,2012-01-02 15:28:39.244,10000,10982.0,W_Completeren aanvraag,START,2012-01-03 18:05:28.583,...,7,W_Completeren aanvraag,W_Completeren aanvraag,A_ACCEPTED,2012-01-02 19:46:24.381,2012-01-03 18:12:38.327,429.744,80344.202,8.0,4170.680987
7,7,23,32422708117511,197219,2012-01-02 15:28:39.244,10000,10982.0,A_ACCEPTED,COMPLETE,2012-01-03 18:12:38.327,...,8,W_Completeren aanvraag,W_Completeren aanvraag,O_SELECTED,2012-01-03 18:05:28.583,2012-01-03 18:15:34.782,176.455,429.744,9.0,14371.812884
8,8,24,32422708117513,197219,2012-01-02 15:28:39.244,10000,10982.0,O_SELECTED,COMPLETE,2012-01-03 18:15:34.782,...,9,A_ACCEPTED,W_Completeren aanvraag,A_FINALIZED,2012-01-03 18:12:38.327,2012-01-03 18:15:34.782,0.000,176.455,10.0,10105.820106
9,9,25,32422708117512,197219,2012-01-02 15:28:39.244,10000,10982.0,A_FINALIZED,COMPLETE,2012-01-03 18:15:34.782,...,10,O_SELECTED,A_ACCEPTED,O_CREATED,2012-01-03 18:15:34.782,2012-01-03 18:15:35.911,1.129,0.000,11.0,13081.201623


In [4]:
train2 = train[train['position'] == 1]
test2 = test[test['position'] == 1]
train2.head(10)

,Unnamed: 0,index,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,...,time_of_day,position,prev_event,2prev_event,next_event,prev_time,next_time,seconds_next,seconds_prev,next position
0,0,0,0,173688,2011-10-01 00:38:44.546,20000,112.0,A_SUBMITTED,COMPLETE,2011-10-01 00:38:44.546,...,0,1,FIRST EVENT,FIRST EVENT,A_PARTLYSUBMITTED,2011-10-01 00:38:44.546,2011-10-01 00:38:44.880,0.334,0.0,2.0
26,26,4,4294967296,173691,2011-10-01 08:08:58.256,5000,112.0,A_SUBMITTED,COMPLETE,2011-10-01 08:08:58.256,...,8,1,FIRST EVENT,FIRST EVENT,A_PARTLYSUBMITTED,2011-10-01 08:08:58.256,2011-10-01 08:09:02.195,3.939,0.0,2.0
65,65,8,8589934592,173694,2011-10-01 08:10:30.287,7000,112.0,A_SUBMITTED,COMPLETE,2011-10-01 08:10:30.287,...,8,1,FIRST EVENT,FIRST EVENT,A_PARTLYSUBMITTED,2011-10-01 08:10:30.287,2011-10-01 08:10:30.591,0.304,0.0,2.0
124,124,10,12884901888,173697,2011-10-01 08:11:08.865,15000,112.0,A_SUBMITTED,COMPLETE,2011-10-01 08:11:08.866,...,8,1,FIRST EVENT,FIRST EVENT,A_PARTLYSUBMITTED,2011-10-01 08:11:08.866,2011-10-01 08:11:09.035,0.169,0.0,2.0
127,127,15,17179869184,173700,2011-10-01 08:15:39.894,5000,112.0,A_SUBMITTED,COMPLETE,2011-10-01 08:15:39.894,...,8,1,FIRST EVENT,FIRST EVENT,A_PARTLYSUBMITTED,2011-10-01 08:15:39.894,2011-10-01 08:15:40.102,0.208,0.0,2.0
130,130,18,21474836480,173703,2011-10-01 09:45:25.806,13500,112.0,A_SUBMITTED,COMPLETE,2011-10-01 09:45:25.806,...,9,1,FIRST EVENT,FIRST EVENT,A_PARTLYSUBMITTED,2011-10-01 09:45:25.806,2011-10-01 09:45:25.981,0.175,0.0,2.0
139,139,20,25769803776,173706,2011-10-01 09:45:37.274,18000,112.0,A_SUBMITTED,COMPLETE,2011-10-01 09:45:37.274,...,9,1,FIRST EVENT,FIRST EVENT,A_PARTLYSUBMITTED,2011-10-01 09:45:37.274,2011-10-01 09:45:37.363,0.089,0.0,2.0
153,153,25,30064771072,173709,2011-10-01 09:57:42.994,11000,112.0,A_SUBMITTED,COMPLETE,2011-10-01 09:57:42.994,...,9,1,FIRST EVENT,FIRST EVENT,A_PARTLYSUBMITTED,2011-10-01 09:57:42.994,2011-10-01 09:57:43.182,0.188,0.0,2.0
165,165,29,34359738368,173712,2011-10-01 09:58:30.533,30000,112.0,A_SUBMITTED,COMPLETE,2011-10-01 09:58:30.533,...,9,1,FIRST EVENT,FIRST EVENT,A_PARTLYSUBMITTED,2011-10-01 09:58:30.533,2011-10-01 09:58:30.678,0.145,0.0,2.0
179,179,32,38654705664,173715,2011-10-01 09:59:10.501,45000,112.0,A_SUBMITTED,COMPLETE,2011-10-01 09:59:10.501,...,9,1,FIRST EVENT,FIRST EVENT,A_PARTLYSUBMITTED,2011-10-01 09:59:10.501,2011-10-01 09:59:10.789,0.288,0.0,2.0


In [5]:
def calculate_metrics(df):
    df['timestamp_prediction'] = df['Prediction time next event'].values.astype(np.int64) // 10 ** 9
    
    y_pred = []
    y_true = []
    for i in range(df.shape[0] - 1):
        if df.iloc[i]['case concept:name'] == df.iloc[i + 1]['case concept:name']:
            y_pred.append(df.iloc[i]['timestamp_prediction'])
            y_true.append(df.iloc[i + 1]['timestamp_finish'])
            
    print('Mean Absolute Error:', round(mean_absolute_error(y_true, y_pred)/3600,3))
    print('Root Mean Squared Error:', round(np.sqrt(mean_squared_error(y_true, y_pred)/3600),3))
    print('R2 score:', round(r2_score(y_true, y_pred),3))
    
    
calculate_metrics(test) 


KeyError: 'Prediction time next event'

In [6]:
def MAE(test):
    test2 = test.dropna(subset=['prediction_seconds','seconds_next'])
    
    y_pred = test2['prediction_seconds']
    y_true = test2['seconds_next']
    
    return mean_absolute_error(y_true, y_pred)

In [7]:
MAE(test)

74668.08540604301